# Guided Project: Analyzing Kickstarter Projects
[Source](https://app.dataquest.io/c/150/m/811/guided-project%3A-analyzing-kickstarter-projects/1/retrieving-column-data-types)

For this guided project, you'll take on the role of a data analyst at a startup. The product team is considering launching a campaign on Kickstarter to test the viability of some offerings. You've been asked to pull data that will help the team understand what might influence the success of a campaign. The data source is a selection of fields from Kaggle.

Specifically, we'll answer the following questions:

What types of projects are most likely to be successful?
Which projects fail?
To get you started, here are the definitions of the columns in this data:

```sql
ID: Kickstarter project ID
name: Name of project
category: Category of project
main_category: Main category of project
goal: Fundraising goal
pledged: Amount pledged
state: State of project (successful, canceled, etc.)
backers: Number of project backers
```

___
## Establish the database connection

In [12]:
# since the dataset is already pre-loaded, to have it run here I am going to pre-load it in using the below script

import pandas as pd
import sqlite3
from IPython.display import display

# File path to the CSV
csv_path = "D:/GitHub/important-reference-repo/Data/ks-projects-201612.csv"

try:
    # 1. Read the CSV with correct encoding and handle trailing spaces
    df = pd.read_csv(
        csv_path,
        encoding='windows-1252',
        skipinitialspace=True  # This handles trailing/leading spaces in column names
    )
    
    # Clean column names by stripping whitespace
    df.columns = df.columns.str.strip()
    print("Cleaned column names:", list(df.columns))
    
    # 2. Select and rename the columns you want
    column_mapping = {
        'ID': 'id',
        'name': 'name',
        'category': 'category',
        'main_category': 'main_category',
        'goal': 'goal',
        'pledged': 'pledged',
        'state': 'state',
        'backers': 'backers'
    }
    
    # Keep only the columns we want and rename them
    df = df[list(column_mapping.keys())].rename(columns=column_mapping)
    
    print("\nFirst 3 rows of selected data:")
    display(df.head(3))
    
    # 3. Create SQLite database
    db_path = "ksprojects.db"
    conn = sqlite3.connect(db_path)
    
    # Define SQL data types
    dtype_mapping = {
        'id': 'INTEGER PRIMARY KEY',
        'name': 'TEXT',
        'category': 'TEXT',
        'main_category': 'TEXT',
        'goal': 'REAL',
        'pledged': 'REAL',
        'state': 'TEXT',
        'backers': 'INTEGER'
    }
    
    # Create table
    df.to_sql(
        'ksprojects',
        conn,
        if_exists='replace',
        index=False,
        dtype=dtype_mapping
    )
    
    print(f"\n✅ Successfully created SQLite database at: {db_path}")
    print("\nTable structure:")
    display(pd.read_sql("PRAGMA table_info(ksprojects);", conn))
    
except Exception as e:
    print(f"\n❌ Error: {str(e)}")
    if 'df' in locals():
        print("\nPartial data loaded:")
        display(df.head(3))
finally:
    if 'conn' in locals():
        conn.close()

# Verification function
def query_db(query="SELECT * FROM ksprojects LIMIT 5"):
    with sqlite3.connect(db_path) as conn:
        return pd.read_sql(query, conn)

# Test the database
print("\nTesting database query...")
display(query_db())

Cleaned column names: ['ID', 'name', 'category', 'main_category', 'currency', 'deadline', 'goal', 'launched', 'pledged', 'state', 'backers', 'country', 'usd pledged', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16']

First 3 rows of selected data:


C:\Users\carol\AppData\Local\Temp\ipykernel_26228\3177498674.py:12: DtypeWarning: Columns (13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


,id,name,category,main_category,goal,pledged,state,backers
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,1000,0,failed,0
1,1000004038,Where is Hank?,Narrative Film,Film & Video,45000,220,failed,3
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,5000,1,failed,1



✅ Successfully created SQLite database at: ksprojects.db

Table structure:


,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,name,TEXT,0,None,0
2,2,category,TEXT,0,None,0
3,3,main_category,TEXT,0,None,0
4,4,goal,REAL,0,None,0
5,5,pledged,REAL,0,None,0
6,6,state,TEXT,0,None,0
7,7,backers,INTEGER,0,None,0



Testing database query...


,id,name,category,main_category,goal,pledged,state,backers
0,5971,An American Apocalypse: The Reckoning,Comics,Comics,48000.0,376.0,failed,7
1,18520,Grandma's are Life,World Music,Music,15000.0,62.0,failed,4
2,21109,Meta,Performance Art,Art,150.0,173.0,successful,11
3,21371,Water Powered Car,Technology,Technology,75000.0,120.0,failed,4
4,24380,Puss N' Books: A relaxing cat cafe and bookstore.,Spaces,Food,20000.0,776.0,failed,18


In [13]:
# Install the ipython-sql package if you haven't already
!pip install ipython-sql
!pip3 install ipython-sql
# Load the SQL extension
%load_ext sql


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


The sql extension is already loaded. To reload it, use:
  %reload_ext sql



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
# First, let's fix the prettytable configuration
!pip install --upgrade prettytable
!pip install --upgrade ipython-sql

# Then restart your notebook kernel (Kernel -> Restart Kernel)


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
# Connect to your SQLite database
%load_ext sql
%sql sqlite:///ksprojects.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [16]:
import sqlite3
from IPython.display import display
import pandas as pd

# Connect to the database
conn = sqlite3.connect('ksprojects.db')

# Safely handle database connections
def sql_query(query):
    with sqlite3.connect('ksprojects.db') as conn:
        df = pd.read_sql(query, conn)
        # display(df) # it was showing the query results as duplicates by including this too
        return df

# Example usage:
sql_query("""
SELECT 
    category,
    COUNT(*) as total_projects,
    SUM(CASE WHEN state = 'successful' THEN 1 ELSE 0 END) as successful
FROM ksprojects
GROUP BY category
ORDER BY successful DESC
LIMIT 10;
""")

,category,total_projects,successful
0,Shorts,11681,6315
1,Product Design,17477,5919
2,Tabletop Games,10708,5731
3,Music,13907,5559
4,Documentary,14891,5454
5,Theater,6833,4269
6,Indie Rock,5372,3438
7,Rock,6345,3275
8,Food,10533,3094
9,Art,6894,2784


# Instructions
This database consists of one table, `ksprojects`.

1. List the column names and data types for the ksprojects table in the database using `PRAGMA table_info`

In [17]:
sql_query("""
PRAGMA table_info(ksprojects)
""")

,cid,name,type,notnull,dflt_value,pk
0,0,id,INTEGER,0,None,1
1,1,name,TEXT,0,None,0
2,2,category,TEXT,0,None,0
3,3,main_category,TEXT,0,None,0
4,4,goal,REAL,0,None,0
5,5,pledged,REAL,0,None,0
6,6,state,TEXT,0,None,0
7,7,backers,INTEGER,0,None,0


# Initial Selection of Rows and Columns
We are now going to begin constructing a query with several components. Each screen of this project will guide you through another piece, until you have the final query.

To start, even a database with one table likely contains data that's not necessary to our analysis. In this step, we'll specify which columns to read from the table.

## Instructions
1. Pull the relevant columns from the `ksprojects` table that will allow us to assess a project's result based on its main category, amount of money set as a goal, number of backers, and amount of money pledged. Return just the first 10 rows.


In [18]:
sql_query("""
SELECT main_category AS 'Main Category',
    goal AS 'Amount of Money Set as Goal',
    backers AS 'Number of Backers',
    pledged AS 'Amount of Money Pledged'
FROM ksprojects
LIMIT 10;
""")

,Main Category,Amount of Money Set as Goal,Number of Backers,Amount of Money Pledged
0,Comics,48000.0,7,376.0
1,Music,15000.0,4,62.0
2,Art,150.0,11,173.0
3,Technology,75000.0,4,120.0
4,Food,20000.0,18,776.0
5,Fashion,18000.0,42,2798.0
6,Food,2500.0,68,3239.0
7,Film & Video,10500.0,66,11545.0
8,Art,20000.0,171,21563.0
9,Film & Video,25000.0,505,66237.0


# Filtering by Category
From this point on in your project each screen will build upon the existing query. To help save time, we will provide the suggested answer code from the previous screen in the coding environment, but if you would prefer to use your own code feel free to copy and paste it!

Now that you've selected the relevant columns, we'll filter the data to include only those in certain categories.

Thomas Edison famously said, "I have not failed. I've just found 10,000 ways that won't work." To that end, your product team would like to know more about the projects that weren't successful.

## Instructions
1. Repeat your query from the previous screen, but this time only keep the records where the project state is either `'failed'`, `'canceled'`, or `'suspended'`.

In [19]:
sql_query("""
SELECT main_category, goal, backers, pledged
FROM ksprojects
WHERE state IN ('failed', 'canceled', 'suspended')
LIMIT 10;
""")

,main_category,goal,backers,pledged
0,Comics,48000.0,7,376.0
1,Music,15000.0,4,62.0
2,Technology,75000.0,4,120.0
3,Food,20000.0,18,776.0
4,Fashion,18000.0,42,2798.0
5,Art,10000.0,0,0.0
6,Games,65000.0,2,11.0
7,Film & Video,5000.0,3,135.0
8,Technology,100000.0,0,0.0
9,Music,1800.0,1,15.0


# Filtering by Quantity

For our analysis, we'll only want to look at projects of a certain size. This is because there are a lot of small projects in the database that aren't relevant to our analysis. Now that we've filtered our records to meet certain categories, let's also filter them to meet given quantities, too.

## Instructions
1. Expand your query from the previous screen to find which of these projects had at least 100 backers and at least $20,000 pledged.

In [21]:
sql_query("""
SELECT name, main_category, backers, pledged, goal, state
 FROM ksprojects
WHERE state IN ('failed', 'canceled', 'suspended') AND
    backers >= 100 AND
    pledged >= 20000
LIMIT 10;
""")

,name,main_category,backers,pledged,goal,state
0,"LUCI™, Advanced Lucid Dream Inducer (Canceled)",Technology,2569,363302.16,40000.0,canceled
1,HOLHO - Hologram generator for SMARTPHONE and ...,Design,319,26943.00,58000.0,canceled
2,WarQuest by Glenn Drover (Canceled),Games,286,60583.00,85000.0,canceled
3,Online Educational Gaming Ecosystem for Children,Technology,158,62989.00,100000.0,failed
4,Help John Start QuadshoX,Technology,218,54681.00,96000.0,failed
5,New Brooklyn Theater,Theater,510,41412.01,200000.0,failed
6,Yellowberry Swim - Active Swimwear for Young G...,Fashion,362,33553.00,50000.0,canceled
7,Merry Go Round (Canceled),Games,129,33710.00,110000.0,canceled
8,Shift: The New Generation of Drone and Controller,Technology,272,103831.00,50000.0,canceled
9,"TapTap, a touch communication wristband",Technology,890,107148.74,130000.0,failed


# Ordering Results
In addition to selecting the relevant columns and filtering the relevant rows, sorting your results can be quite valuable in making sense of the data. In this case, the product team would like to view projects by categories, along with the percentage of the goal that was funded.

## Instructions
1. Continue building on your query from the previous screen. This time, you'll sort the results by two fields:
- Main category sorted in ascending order.
- A calculated field called `pct_pledged`, which divides `pledged` by `goal`. Sort this field in descending order. (Add `pct_pledged` to the `SELECT` clause, too.)
2. Now, modify your query so that only projects in a `failed` state are returned.

In [22]:
sql_query("""
SELECT main_category, backers, pledged, goal,
        pledged / goal AS pct_pledged
  FROM ksprojects
 WHERE state = 'failed'
   AND backers >= 100 AND pledged >= 20000
ORDER BY main_category ASC, pct_pledged DESC
 LIMIT 10;
""")

,main_category,backers,pledged,goal,pct_pledged
0,Art,294,52565.25,60000.0,0.876088
1,Art,172,21310.00,25000.0,0.852400
2,Art,185,27630.24,35000.0,0.789435
3,Art,552,37621.97,50000.0,0.752439
4,Art,336,33485.00,52000.0,0.643942
5,Art,130,20374.99,35000.0,0.582143
6,Art,133,54387.00,100000.0,0.543870
7,Art,136,39206.49,77777.0,0.504088
8,Art,257,50230.00,100000.0,0.502300
9,Art,115,27413.00,80000.0,0.342662


# Applying Conditional Logic

It can often be helpful to make sense of a set of records by grouping them into categories based on some condition, which in SQL can be done with `CASE` statements.

While it's interesting to view the results of failed projects by metrics like the number of backers, what really makes or breaks a Kickstarter project is whether it meets its pledge goal.

## Instructions
1. Create a field `funding_status` that applies the following logic based on the percentage of amount pledged to campaign goal:
- If the percentage pledged is greater than or equal to 1, then the project is "Fully funded".
- If the percentage pledged is between 75% and 100%, then the project is "Nearly funded".
- If the percentage pledged is less than 75%, then the project is "Not nearly funded".
2. Write either a line or block comment with your observations about the funding status of the sample output. For example, are these failed projects failing because they don't have any backers or funding?

In [23]:
sql_query("""
  SELECT main_category, backers, pledged, goal,
         pledged / goal AS pct_pledged,
     CASE WHEN (pledged / goal) >= 1 THEN 'Fully funded'
          WHEN (pledged / goal) BETWEEN .75 AND 1 THEN 'Nearly funded'
          ELSE 'Not nearly funded'
     END AS funding_status
    FROM ksprojects
   WHERE state IN ('failed')
     AND backers >= 100 AND pledged >= 20000
ORDER BY main_category, pct_pledged DESC
   LIMIT 10;
-- yes, these failed projects are failing because they don't have enough funding
""")

,main_category,backers,pledged,goal,pct_pledged,funding_status
0,Art,294,52565.25,60000.0,0.876088,Nearly funded
1,Art,172,21310.00,25000.0,0.852400,Nearly funded
2,Art,185,27630.24,35000.0,0.789435,Nearly funded
3,Art,552,37621.97,50000.0,0.752439,Nearly funded
4,Art,336,33485.00,52000.0,0.643942,Not nearly funded
5,Art,130,20374.99,35000.0,0.582143,Not nearly funded
6,Art,133,54387.00,100000.0,0.543870,Not nearly funded
7,Art,136,39206.49,77777.0,0.504088,Not nearly funded
8,Art,257,50230.00,100000.0,0.502300,Not nearly funded
9,Art,115,27413.00,80000.0,0.342662,Not nearly funded


In [24]:
# close the connection when we are done
conn.close()